In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [19]:
!unzip test.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: test/test_03897.jpg     
  inflating: __MACOSX/test/._test_03897.jpg  
  inflating: test/test_02551.jpg     
  inflating: __MACOSX/test/._test_02551.jpg  
  inflating: test/test_04120.jpg     
  inflating: __MACOSX/test/._test_04120.jpg  
  inflating: test/test_07429.jpg     
  inflating: __MACOSX/test/._test_07429.jpg  
  inflating: test/test_10155.jpg     
  inflating: __MACOSX/test/._test_10155.jpg  
  inflating: test/test_00346.jpg     
  inflating: __MACOSX/test/._test_00346.jpg  
  inflating: test/test_01058.jpg     
  inflating: __MACOSX/test/._test_01058.jpg  
  inflating: test/test_06737.jpg     
  inflating: __MACOSX/test/._test_06737.jpg  
  inflating: test/test_09404.jpg     
  inflating: __MACOSX/test/._test_09404.jpg  
  inflating: test/test_10141.jpg     
  inflating: __MACOSX/test/._test_10141.jpg  
  inflating: test/test_00352.jpg     
  inflating: __MACOSX/test/._test_00352.jpg  
  inflating: test/test_06723.jpg     
  

In [18]:
cd image

/content/MyDrive/MyDrive/BoAZ/image


In [12]:
!rename ._test test test_?????

syntax error at line 1, near "{
.", in:
._test


In [15]:
!ls

In [2]:
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00


In [3]:
cd /content/MyDrive/MyDrive/BoAZ


/content/MyDrive/MyDrive/BoAZ


# Dataset

In [20]:
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
from tqdm import tqdm_notebook as tqdm
import warnings
import joblib

df_train = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/train.csv')

df_train.head()

,ID,image_id,question,answer
0,TRAIN_000000,train_000000,Is this in the wild or zoo?,zoo
1,TRAIN_000001,train_000001,Is this car a recent model?,yes
2,TRAIN_000002,train_000002,Is the man being safe?,yes
3,TRAIN_000003,train_000003,What are the walls made from?,drywall
4,TRAIN_000004,train_000004,How many players are sitting on the bench?,8


In [21]:
df_test = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/test.csv')
df_test.head()

,ID,image_id,question
0,TEST_00000,test_00000,Is this a trade show?
1,TEST_00001,test_00001,Where are the people?
2,TEST_00002,test_00002,What color is the ski suit?
3,TEST_00003,test_00003,What color are the cabinets?
4,TEST_00004,test_00004,Is there a person standing in the background?


In [20]:
test_images = glob("/content/MyDrive/MyDrive/BoAZ/image/test/*.jpg")
len(test_images)

11915

In [21]:
len(df_train)

359521

In [22]:
train_images = glob("/content/MyDrive/MyDrive/BoAZ/image/train/*.jpg")
len(train_images)

53156

In [23]:
train_images_indrive = []
for i in range(len(train_images)):
    train_images_indrive.append(train_images[i][42:-4])
print(len(train_images_indrive),train_images_indrive[:5])

53156 ['train_077676', 'train_001039', 'train_101231', 'train_068160', 'train_023456']


In [32]:
filtered_df = df_train[df_train['image_id'].isin(train_images_indrive)]
len(filtered_df)

178650

In [33]:
yesorno = ["yes", "no"]
filtered_df = filtered_df[~filtered_df['answer'].isin(yesorno)]
#filtered_df.reset_index(inplace=True,drop=True)
filtered_df.head()
len(filtered_df)
#filtered_df.to_csv(path_or_buf="without_closed_ended_question_index.csv")

111095

In [ ]:
len(filtered_df)

223926

In [ ]:
filtered_df.to_csv(path_or_buf="without_closed_ended_question.csv")

In [ ]:
type(filtered_df['image_id'].value_counts())

pandas.core.series.Series

In [ ]:
filtered_df.iloc[14814]

ID                   TRAIN_014814
image_id             train_013397
question    Is the beach crowded?
answer                         no
Name: 14814, dtype: object

In [ ]:
filtered_df.isna().value_counts()

ID     image_id  question  answer
False  False     False     False     359521
dtype: int64

In [ ]:
filtered_df

In [ ]:
filtered_df.drop(columns = ['ID'])

In [ ]:
counts_df = filtered_df['image_id'].value_counts().to_frame()
counts_df.head()

,image_id
train_001939,104
train_004938,60
train_000107,60
train_000840,54
train_002746,51


In [ ]:
counts_df

In [ ]:
len(filtered_df)

80703

In [ ]:
filtered_df.describe()

,ID,image_id,question,answer
count,80703,80703,80703,80703
unique,80703,24083,53782,7533
top,TRAIN_000004,train_004938,What room is this?,yes
freq,1,108,214,15434


In [ ]:
filtered_df['image_id'].value_counts()

train_004938    108
train_002746     93
train_002289     66
train_011330     57
train_013137     56
               ... 
train_057421      1
train_057430      1
train_088594      1
train_057462      1
train_107226      1
Name: image_id, Length: 24083, dtype: int64

In [ ]:
filtered_df[filtered_df['image_id']=='train_004938']

,ID,image_id,question,answer
5136,TRAIN_005136,train_004938,How many people are in the room?,2
5942,TRAIN_005942,train_004938,Are they running towards the TV?,no
11225,TRAIN_011225,train_004938,How many lamps are in the room?,0
13767,TRAIN_013767,train_004938,Is someone sitting on the floor?,no
14464,TRAIN_014464,train_004938,How many people are in the photo?,2
...,...,...,...,...
327662,TRAIN_327662,train_004938,Will he win?,yes
330433,TRAIN_330433,train_004938,What type of remotes are they?,wii
330591,TRAIN_330591,train_004938,Are there any children?,no
332977,TRAIN_332977,train_004938,Is it daytime?,no


**한장의 사진에 여러가지 질문과 답변을 했다**

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset


class VQADataset(Dataset):
    def __init__(self, df, processor, mode='train'):
        self.df = df
        self.mode = mode

        self.vis_processor = processor.image_processor
        self.txt_processor = processor.tokenizer

        if mode == 'test':
            self.img_path = 'data/image/test'
        else:
            self.img_path = 'data/image/train'

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg')
        image = Image.open(img_name).convert('RGB')

        inputs = self.txt_processor(
            text=row['question'],
            max_length=32,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_tensors="pt",
        )
        inputs.update(
            self.vis_processor(
                images=image,
                return_tensors="pt",
            )
        )

        if self.mode != 'test':
            inputs["labels"] = self.txt_processor(
                text=row['answer'],
                max_length=32,
                padding='max_length',
                return_tensors="pt",
            ).input_ids

        for k in inputs.keys():
            inputs[k] = inputs[k].squeeze()

        return inputs

In [6]:
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers.trainer_utils import seed_worker
from transformers import BlipConfig, BlipProcessor, BlipImageProcessor, BertTokenizerFast, BlipForQuestionAnswering


In [ ]:
##class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )

image_processor = BlipImageProcessor()
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
processor = BlipProcessor(image_processor=image_processor, tokenizer=tokenizer)
model_config = BlipConfig()
model = BlipForQuestionAnswering(model_config).to(device)

In [ ]:
filtered_df.iloc[0]

ID                                        TRAIN_000004
image_id                                  train_000004
question    How many players are sitting on the bench?
answer                                               8
Name: 4, dtype: object

In [ ]:
# state_dict = torch.load(args.pretrained_ckpt)['model']
# for key in state_dict.copy():
#     value = state_dict.pop(key)
#     if key == 'visual_encoder.pos_embed':
#         value = utils.interpolate_pos_embed(value, model.vision_model)
#     renamed_key = utils.rename_key(key)
#     state_dict[renamed_key] = value
# model.load_state_dict(state_dict, strict=False)

# model.gradient_checkpointing_enable()
train_set = VQADataset(train_df, processor, mode='train')
train_loader = DataLoader(
    train_set,
    batch_size=args.batch_size,
    shuffle=True,
    drop_last=True,
    **loader_dict
)

optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=int(total_steps * args.warmup_ratio),
        num_training_steps=total_steps,
)
scaler = GradScaler()

In [ ]:
def train(epoch, model, loader, optimizer, scheduler, scaler, args, log_freq=1000):
    start = end = time.time()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    model.train()
    optimizer.zero_grad()

    if args.freeze_image_encoder:
        for name, param in model.vision_model.named_parameters():
            param.requires_grad = False
    len_loader = len(loader)
    for i, inputs in enumerate(loader):
        data_time.update(time.time() - end)

        with autocast():
            for k in inputs.keys():
                inputs[k] = inputs[k].to(args.device)

            outputs = model(**inputs)

            loss = outputs.loss / args.grad_accum
            losses.update(loss.item())
        scaler.scale(loss).backward()
        if (i+1) % args.grad_accum == 0 or (i+1) == len_loader:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0 * args.grad_accum)
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()
        batch_time.update(time.time() - end)
        end = time.time()

In [ ]:
valid_acc = 0
    for epoch in range(1, args.num_epochs + 1):
        print('-' * 10)
        print(f'Epoch {epoch} / {args.num_epochs}')

        train_loss = train(epoch, model, train_loader, optimizer, scheduler, scaler, args)

        if args.fold != -1:
            valid_acc = evaluation(model, valid_loader, processor, args, args.device)

        if epoch % args.save_freq == 0:
            file_name = f'epoch{epoch}_acc{valid_acc}.pt'
            torch.save(model.state_dict(), osp.join(args.work_dir_exp, file_name))

        print(f'[Epoch {epoch}] [Train] Loss:{train_loss}')
        if args.fold != -1:
            print(f'[Epoch {epoch}] [Valid] Acc:{valid_acc}')

# Baseline

## Import

In [ ]:
!pip install transformers tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 113.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00


In [ ]:
!pip install tqdm

In [ ]:
import os
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models # 이미지
from torchvision import transforms
from PIL import Image

#from transformers import GPT2Tokenizer, GPT2Model # 텍스트

from tqdm.auto import tqdm

## Dataset

In [ ]:
class VQADataset(Dataset):
    def __init__(self, df, tokenizer, transform, img_path, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.transform = transform
        self.img_path = img_path
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg') # 이미지
        image = Image.open(img_name).convert('RGB')
        image = self.transform(image)

        question = row['question'] # 질문
        question = self.tokenizer.encode_plus(
            question,
            truncation=True,
            add_special_tokens=True,
            max_length=32,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        if not self.is_test:
            answer = row['answer'] # 답변
            answer = self.tokenizer.encode_plus(
                answer,
                max_length=32,
                padding='max_length',
                truncation=True,
                return_tensors='pt')
            return {
                'image': image.squeeze(),
                'question': question['input_ids'].squeeze(),
                'answer': answer['input_ids'].squeeze()
            }
        else:
            return {
                'image': image,
                'question': question['input_ids'].squeeze(),
            }

## Model

In [ ]:
class VQAModel(nn.Module):
    def __init__(self, vocab_size):
        super(VQAModel, self).__init__()
        self.vocab_size = vocab_size

        self.resnet = models.resnet50(pretrained=True)
        self.gpt2 = GPT2Model.from_pretrained('gpt2')
        self.gpt2.resize_token_embeddings(vocab_size) # 추가한 [PAD] 토큰 반영

        combined_features_size = 1000 + self.gpt2.config.hidden_size # resnet 출력 차원 + gpt2 출력 차원
        self.classifier = nn.Linear(combined_features_size, vocab_size)

    def forward(self, images, question):
        image_features = self.resnet(images)
        image_features = image_features.view(image_features.size(0),-1)

        outputs = self.gpt2(question)
        output_features = outputs.last_hidden_state # [batch, sequence, hidden]

        image_features = image_features.unsqueeze(1).expand(-1, output_features.size(1),-1) # [batch, sequence, 1000]

        combined = torch.cat([image_features, output_features], dim=-1) # [batch, sequence, 1000+hidden]
        output = self.classifier(combined) # [batch, vocab_size]
        return output

## DataLoader

In [ ]:
# 데이터 불러오기
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
train_img_path = './image/train'
test_img_path = './image/test'

# dataset & dataloader
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
vocab_size = len(tokenizer)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = VQADataset(train_df, tokenizer, transform, train_img_path, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

## Train & Inference

In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for data in tqdm(loader, total=len(loader)):
        images = data['image'].to(device,torch.float16)
        question = data['question'].to(device,torch.float16)
        answer = data['answer'].to(device,torch.float16)

        optimizer.zero_grad()

        outputs = model(images, question)

        # output: [batch, sequence, vocab], answer : [batch, sequence]
        loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(loader)
    return avg_loss

In [ ]:
def inference(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for data in tqdm(loader, total=len(loader)):
            images = data['image'].to(device,torch.float16)
            question = data['question'].to(device,torch.float16)

            outputs = model(images, question) # [batch, sequence, vocab]

            _, pred = torch.max(outputs, dim=2) # values, indices = _, pred
            preds.extend(pred.cpu().numpy())

    return preds

## Run!

In [ ]:
# device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"current device is {device}")

# Model
model = VQAModel(vocab_size).to(device,torch.float16)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(1):
    avg_loss = train(model, train_loader, optimizer, criterion)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

current device is cuda


  0%|          | 0/5618 [00:00<?, ?it/s]

RuntimeError: ignored

## Post-Processing

In [ ]:
# Dataset & DataLoader
test_dataset = VQADataset(test_df, tokenizer, transform, test_img_path, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# inference
preds = inference(model, test_loader)

no_pad_output = []
for pred in preds:
    output = pred[pred != 50257] # [PAD] token 제외
    no_pad_output.append(tokenizer.decode(output).strip()) # 토큰 id -> 토큰

  0%|          | 0/633 [00:00<?, ?it/s]

RuntimeError: ignored

## Submission

In [ ]:
sample_submission['answer'] = no_pad_output
sample_submission.to_csv('submission.csv', index=False)

In [ ]:
solution = pd.read_csv('solution.csv')

---

# BLIP v2

In [5]:
filtered_train = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/without_closed_ended_question.csv')
filtered_train.head()

,Unnamed: 0,ID,image_id,question,answer
0,0,TRAIN_000000,train_000000,Is this in the wild or zoo?,zoo
1,1,TRAIN_000003,train_000003,What are the walls made from?,drywall
2,2,TRAIN_000004,train_000004,How many players are sitting on the bench?,8
3,3,TRAIN_000006,train_000006,Where is the landing gear?,under plane
4,4,TRAIN_000007,train_000007,What colors are the sheets?,red


In [8]:
train_images = glob("/content/MyDrive/MyDrive/BoAZ/image/train/*.jpg")
len(train_images)
train_images_indrive = []
for i in range(len(train_images)):
    train_images_indrive.append(train_images[i][42:-4])
print(len(train_images_indrive),train_images_indrive[:5])

0 []


In [7]:
len(train_images_indrive)

0

In [ ]:
filtered_train = filtered_df[filtered_df['image_id'].isin(train_images_indrive)]
filtered_train.head()

,ID,image_id,question,answer
0,TRAIN_000000,train_000000,Is this in the wild or zoo?,zoo
1,TRAIN_000001,train_000001,Is this car a recent model?,yes
2,TRAIN_000002,train_000002,Is the man being safe?,yes
3,TRAIN_000003,train_000003,What are the walls made from?,drywall
4,TRAIN_000004,train_000004,How many players are sitting on the bench?,8


In [ ]:
len(filtered_df)

359521

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset


class VQADataset(Dataset):
    def __init__(self, df, processor, mode='train'):
        self.df = df
        self.mode = mode

        self.vis_processor = processor.image_processor
        self.txt_processor = processor.tokenizer

        if mode == 'test':
            self.img_path = 'data/image/test'
        else:
            self.img_path = '/content/MyDrive/MyDrive/BoAZ/image/train'

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg')
        image = Image.open(img_name).convert('RGB')

        inputs = self.txt_processor(
            text=row['question'],
            max_length=32,
            padding='max_length',
            truncation=True,
            return_token_type_ids=False,
            return_tensors="pt",
        )
        inputs.update(
            self.vis_processor(
                images=image,
                return_tensors="pt",
            )
        )

        if self.mode != 'test':
            inputs["labels"] = self.txt_processor(
                text=row['answer'],
                max_length=32,
                padding='max_length',
                return_tensors="pt",
            ).input_ids

        for k in inputs.keys():
            inputs[k] = inputs[k].squeeze()

        return inputs

In [ ]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")

In [ ]:
from torchvision.transforms import RandomResizedCrop, ColorJitter, Compose

size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)

_transforms = Compose([RandomResizedCrop(size)])

In [ ]:
def transforms(examples):
    images = [_transforms(img.convert("RGB")) for img in examples["image"]]
    examples["pixel_values"] = image_processor(images, do_resize=False, return_tensors="pt")["pixel_values"]
    return examples

In [ ]:
class VQADataset(Dataset):
    def __init__(self, df, tokenizer, transform, img_path, is_test=False):
        self.df = df
        self.tokenizer = tokenizer
        self.transform = transform
        self.img_path = img_path
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_name = os.path.join(self.img_path, row['image_id'] + '.jpg') # 이미지
        pre_image = Image.open(img_name)
        # 이미지 정규화 (0과 1 사이로 스케일링)
        normalized_image = (pre_image - np.min(pre_image)) / (np.max(pre_image) - np.min(pre_image))

        # [0, 255] 범위로 되돌리기
        scaled_image = (normalized_image * 255).astype(np.uint8)
        image = Image.fromarray(scaled_image)
        image = self.transform(image)

        question = row['question'] # 질문
        question = self.tokenizer.encode_plus(
            question,
            truncation=True,
            add_special_tokens=True,
            max_length=32,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        if not self.is_test:
            answer = row['answer'] # 답변
            answer = self.tokenizer.encode_plus(
                answer,
                max_length=32,
                padding='max_length',
                truncation=True,
                return_tensors='pt')
            return {
                'image': image.squeeze(),
                'question': question['input_ids'].squeeze(),
                'answer': answer['input_ids'].squeeze()
            }
        else:
            return {
                'image': image,
                'question': question['input_ids'].squeeze(),
            }

In [ ]:
import torch
from transformers import AutoProcessor, BlipProcessor, BlipForQuestionAnswering
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [ ]:
train_img_path = '/content/MyDrive/MyDrive/BoAZ/image/train'
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train_dataset = VQADataset(filtered_df, processor.tokenizer, transform,train_img_path, is_test=False)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

In [ ]:
text = "what is she doing?"
label = "playing tennis"
inputs = processor(images=image, text=text, return_tensors="pt")
labels = processor(text=label, return_tensors="pt").input_ids

inputs["labels"] = labels
outputs = model(**inputs)
loss = outputs.loss
loss.backward()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)
for data in tqdm(train_dataloader, total=len(train_dataloader)):
  images = data['image']
  question = data['question']
  answer = data['answer']

  inputs = processor(images=images, text=question, return_tensors="pt")
  labels = processor(text=answer, return_tensors="pt").input_ids
  inputs["labels"] = labels
  print(question, answer)

  outputs = model(**inputs)

  print(outputs)
  break

  0%|          | 0/27991 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for data in tqdm(loader, total=len(loader)):
        images = data['image'].to(device,torch.float16)
        question = data['question'].to(device,torch.float16)
        answer = data['answer'].to(device,torch.float16)

        optimizer.zero_grad()

        outputs = model(images, question)

        # output: [batch, sequence, vocab], answer : [batch, sequence]
        loss = criterion(outputs.view(-1, outputs.size(-1)), answer.view(-1))
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(loader)
    return avg_loss

In [ ]:
for idx, batch in enumerate(train_dataloader):
    print(batch['image'])
    break

tensor([[[[ 0.3823,  0.4337,  0.4508,  ...,  0.7419,  0.7077,  0.6734],
          [ 0.3994,  0.4337,  0.4508,  ...,  0.7419,  0.7077,  0.6734],
          [ 0.3994,  0.4508,  0.4679,  ...,  0.7419,  0.7077,  0.6734],
          ...,
          [-0.0458, -0.0458, -0.0116,  ...,  0.7933,  0.7591,  0.7248],
          [-0.0629, -0.0629, -0.0629,  ...,  0.7933,  0.7419,  0.7077],
          [-0.0629, -0.0629, -0.0629,  ...,  0.7933,  0.7419,  0.7077]],

         [[ 0.2052,  0.2577,  0.2752,  ...,  0.6429,  0.6078,  0.5728],
          [ 0.2227,  0.2577,  0.2752,  ...,  0.6429,  0.6078,  0.5728],
          [ 0.2227,  0.2752,  0.2927,  ...,  0.6429,  0.6078,  0.5728],
          ...,
          [-0.4776, -0.4776, -0.4426,  ...,  0.6429,  0.5903,  0.5553],
          [-0.4951, -0.4951, -0.4951,  ...,  0.6429,  0.5903,  0.5378],
          [-0.4951, -0.4951, -0.4951,  ...,  0.6429,  0.5903,  0.5378]],

         [[ 0.2173,  0.2696,  0.2871,  ...,  0.8099,  0.7751,  0.7402],
          [ 0.2348,  0.2696,  

In [ ]:
inputs = processor(images=image, text=text, return_tensors="pt")
labels = processor(text=label, return_tensors="pt").input_ids

inputs["labels"] = labels
outputs = model(**inputs)
loss = outputs.loss
loss.backward()

In [ ]:
import torch

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

model.train()

for epoch in range(5):
  print("Epoch:", epoch)
  for idx, batch in enumerate(train_dataloader):
    input_ids = batch.pop("input_ids").to(device)
    pixel_values = batch.pop("pixel_values").to(device)

    outputs = model(input_ids=input_ids,
                    pixel_values=pixel_values,
                    labels=input_ids)

    loss = outputs.loss

    print("Loss:", loss.item())

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

In [ ]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-flan-t5-xl", torch_dtype=torch.float16
)
model.to(device)
path = "./image/train/train_064020.jpg"
image = Image.open(path)

prompt = "Question: how many dogs are there? Answer:"
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device, torch.float16)

generated_ids = model.generate(**inputs)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(generated_text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
model.to(device)

In [ ]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForQuestionAnswering

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to("cuda")

eat


In [ ]:
path = "/content/test_00453.jpg"

raw_image = Image.open(path).convert('RGB')

question = "what does dog do?"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda")

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForQuestionAnswering

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)
path = "/content/test_01379.jpg"
image = Image.open(path).convert('RGB')
# training
text = "what is she doing?"
label = "playing tennis"
inputs = processor(images=image, text=text, return_tensors="pt")
labels = processor(text=label, return_tensors="pt").input_ids

inputs["labels"] = labels
outputs = model(**inputs)
loss = outputs.loss
loss.backward()

# inference
text = "what is she doing?"
inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model.generate(**inputs)
print(processor.decode(outputs[0], skip_special_tokens=True))

playing tennis


In [ ]:
import numpy as np
import pandas as pd
import operator
import os
import string
import re
import random
import sys
import platform
import inflect
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from transformers import Blip2Processor, Blip2ForConditionalGeneration

from tqdm.auto import tqdm
def convert_word_to_number(word):
    p = inflect.engine()
    try:
        return p.singular_noun(word)
    except:
        return None

device = "cuda" if torch.cuda.is_available() else "cpu"
test_img_path= '/content/MyDrive/MyDrive/BoAZ/image/test'
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")
model.to(device)
sample_submission = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/sample_submission.csv')

final_submission = sample_submission.copy()

for idx, row in tqdm(df_test.iterrows()):
    img_name = os.path.join(test_img_path, row['image_id'] + '.jpg')  # 이미지
    image = Image.open(img_name).convert('RGB')

    prompt =  "Question : {} answer:".format(row['question'])
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    generated_text = convert_word_to_number(generated_text)

    final_submission.loc[final_submission['ID'] == row['ID'], 'answer'] = generated_text

    if idx%100 == 0:
        final_submission.to_csv('/content/MyDrive/MyDrive/submission_zeroshot_blip.csv', index=False)

final_submission.to_csv('/content/MyDrive/MyDrive/submission_zeroshot_blip_no_adjust.csv', index=False)

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored

# etc

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip uninstall dill

Found existing installation: dill 0.3.7
Uninstalling dill-0.3.7:
  Would remove:
    /usr/local/bin/get_gprof
    /usr/local/bin/get_objgraph
    /usr/local/bin/undill
    /usr/local/lib/python3.10/dist-packages/dill-0.3.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/dill/*
Proceed (Y/n)? Y
  Successfully uninstalled dill-0.3.7


In [ ]:
!pip install dill==0.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.4/151.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dill: filename=dill-0.3.0-py3-none-any.whl size=77524 sha256=7d0626ca1779f4f2f123625655859b75f28ad7571d7455eebef8a92d35d8b2ce
  Stored in directory: /root/.cache/pip/wheels/88/56/1a/003b80a0c1679158a75449cc4ef71b86bd7f980db666c1c8ab
Successfully built dill


In [ ]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets[vision]

In [ ]:
from datasets import load_dataset

#dataset = load_dataset("tabtoyou/KoLLaVA-CC3M-Pretrain-595K")

In [ ]:
dataset = load_dataset("ybelkada/football-dataset", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
dataset = load_dataset("tabtoyou/KoLLaVA-CC3M-Pretrain-595K")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = load_dataset("tabtoyou/KoLLaVA-Instruct-150k",data_dir="/content/Mydrive")

EmptyDatasetError: ignored

In [ ]:
dataset[0]['image']

'000000215677.jpg'

In [ ]:
from PIL import Image
import requests
def display_image(image_url):
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    plt.imshow(img)
    plt.axis("off")
    plt.show()
display_image(dataset[0]['image'])

MissingSchema: ignored

In [ ]:
dataset

Dataset({
    features: ['conversations', 'id', 'image'],
    num_rows: 313129
})

In [ ]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': '<image>\n이러한 프리스비 묘기를 하려면 어떤 기술이 필요할까요?'},
  {'from': 'gpt',
   'value': '이미지에서 남성이 다리 사이 또는 다리 밑으로 프리스비를 통과시키는 프리스비 묘기를 수행하려면 여러 가지 기술이 필요합니다. 이러한 기술에는 눈과 손의 협응력, 민첩성, 균형감각, 유연성 및 손재주가 포함됩니다. 또한 몸의 움직임을 제어하면서 프리스비를 정확하게 던지고 잡을 수 있는 능력도 필수적입니다. 기술을 완벽하게 익히려면 이러한 기술을 연습하고 반복을 통해 근육 기억력을 키우는 것이 도움이 될 것입니다.'}],
 'id': '000000215677',
 'image': '000000215677.jpg'}

In [ ]:
dataset[0]['image']

'000000215677.jpg'

In [ ]:
dataset[0]['conversations']

[{'from': 'human', 'value': '<image>\n이러한 프리스비 묘기를 하려면 어떤 기술이 필요할까요?'},
 {'from': 'gpt',
  'value': '이미지에서 남성이 다리 사이 또는 다리 밑으로 프리스비를 통과시키는 프리스비 묘기를 수행하려면 여러 가지 기술이 필요합니다. 이러한 기술에는 눈과 손의 협응력, 민첩성, 균형감각, 유연성 및 손재주가 포함됩니다. 또한 몸의 움직임을 제어하면서 프리스비를 정확하게 던지고 잡을 수 있는 능력도 필수적입니다. 기술을 완벽하게 익히려면 이러한 기술을 연습하고 반복을 통해 근육 기억력을 키우는 것이 도움이 될 것입니다.'}]

In [ ]:
dataset[0]['conversations'][0]['value'][8:]

'이러한 프리스비 묘기를 하려면 어떤 기술이 필요할까요?'

In [ ]:
dataset[0]['conversations'][1]['value']

'이미지에서 남성이 다리 사이 또는 다리 밑으로 프리스비를 통과시키는 프리스비 묘기를 수행하려면 여러 가지 기술이 필요합니다. 이러한 기술에는 눈과 손의 협응력, 민첩성, 균형감각, 유연성 및 손재주가 포함됩니다. 또한 몸의 움직임을 제어하면서 프리스비를 정확하게 던지고 잡을 수 있는 능력도 필수적입니다. 기술을 완벽하게 익히려면 이러한 기술을 연습하고 반복을 통해 근육 기억력을 키우는 것이 도움이 될 것입니다.'

In [ ]:
dataset[1]['conversations'][0]['value'][8:]

'이 여성은 이런 날씨에 걷는 동안 어떤 예방 조치를 취하고 있나요?'

In [ ]:
from PIL import Image
import requests
from transformers import AutoProcessor, BlipForQuestionAnswering

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = AutoProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [ ]:
eng_dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K",split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
eng_dataset

Dataset({
    features: ['conversations', 'id', 'image'],
    num_rows: 394276
})

In [ ]:
eng_dataset[0]['conversations'][0]['value'][8:]

'What skill set might someone need to perform such a frisbee trick?'

In [ ]:
eng_dataset[0]['image']

'000000215677.jpg'

In [ ]:
eng_dataset[0]

{'conversations': [{'from': 'human',
   'value': '<image>\nWhat skill set might someone need to perform such a frisbee trick?'},
  {'from': 'gpt',
   'value': 'To perform the frisbee trick shown in the image, where the man is passing a frisbee between or underneath his legs, a person would need a combination of skills. These skills include good hand-eye coordination, agility, balance, flexibility, and dexterity. Additionally, the ability to throw and catch the frisbee accurately while maintaining control of bodily movements would also be essential. To perfect the trick, practicing these skills and building up muscle memory through repetition would be beneficial.'}],
 'id': '000000215677',
 'image': '000000215677.jpg'}

In [ ]:
for i in range(len(eng_dataset)):
    if '190697' in str(i):
        print(eng_dataset[i])

{'conversations': [{'from': 'human', 'value': 'How many people are there in the boat?\n<image>'}, {'from': 'gpt', 'value': 'There are two people in the red and white boat.'}, {'from': 'human', 'value': 'What does the surrounding environment of the boat look like?'}, {'from': 'gpt', 'value': "The boat is traveling down a canal, which has busy streets on both sides of it, indicating that it's likely in a town or city. Near the canal, there's also a small docking area."}, {'from': 'human', 'value': 'What type of waterway is the boat traveling on?'}, {'from': 'gpt', 'value': 'The boat is traveling on a river, specifically through a bustling waterway in a town.'}, {'from': 'human', 'value': 'What color is the boat?'}, {'from': 'gpt', 'value': 'The boat is red and white.'}, {'from': 'human', 'value': 'Can you give a brief description of the situation in the image?'}, {'from': 'gpt', 'value': 'The image depicts a red and white boat with two people on board traveling down a bustling canal in a

In [ ]:
str(1)

'1'

# vqa blip

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!git clone https://github.com/Dongwoo-Im/dacon_vqa.git

Cloning into 'dacon_vqa'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 9 (delta 1), pack-reused 0
Receiving objects: 100% (9/9), 8.00 KiB | 314.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [ ]:
cd dacon_vqa

/content/MyDrive/MyDrive/BoAZ/dacon_vqa


In [ ]:
!pip install wandb

In [ ]:
!python /content/MyDrive/MyDrive/BoAZ/dacon_vqa/train.py

2023-08-17 08:39:24.124649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/MyDrive/MyDrive/BoAZ/dacon_vqa/train.py", line 237, in <module>
    main(args)
  File "/content/MyDrive/MyDrive/BoAZ/dacon_vqa/train.py", line 168, in main
    state_dict = torch.load('/content/MyDrive/MyDrive/BoAZ/dacon_vqa/model_base.pth')#['model']
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 797, in load
    with _open_zipfile_reader(opened_file) as opened_zipfile:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 283, in __init__
    super().__init__(torch._C.PyTorchFileReader(name_or_buffer))
RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory


# Code from DACON

In [24]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 15.1 MB/s eta 0:00:00


In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models

import transformers
from transformers import AutoProcessor
from transformers import BlipForQuestionAnswering
transformers.logging.set_verbosity_error()

import re
from datetime import datetime
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from argparse import ArgumentParser

# import wandb
# from pytorch_lightning.loggers import WandbLogger
# wandb_logger = WandbLogger(name="BLIP", project="DACON_236118")

parser = ArgumentParser(description="BLIP")
parser.add_argument('--vision_language_pretrained_model', default="Salesforce/blip-vqa-capfilt-large", type=str)
parser.add_argument('--image_size', default=384, type=int)
parser.add_argument('--text_len', default=32, type=int)
parser.add_argument('--optimizer', default="adamw", type=str)
parser.add_argument('--learning_rate', default=0.00001, type=float)
parser.add_argument('--scheduler', default="none", type=str)
parser.add_argument('--batch_size', default=8, type=int)
parser.add_argument('--epochs', default=5, type=int)
parser.add_argument('--validation_size', default=0.1, type=int)
parser.add_argument('--seed', default=826, type=int)
parser.add_argument('--mixed_precision', default=16, type=int)
parser.add_argument('--device', default=[0], type=list)
parser.add_argument('--num_workers', default=0, type=int)
args = parser.parse_args('')

# wandb.config.update(args)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

image_size = args.image_size
text_len = args.text_len
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
VALIDATION_SIZE = args.validation_size
SEED = args.seed

def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(SEED)

set_seeds()

idx = f"{args.vision_language_pretrained_model}_{args.seed}"
idx

INFO:lightning_fabric.utilities.seed:Global seed set to 826


'Salesforce/blip-vqa-capfilt-large_826'

In [25]:
vocab_size = len(processor.tokenizer)
vocab_size

30522

In [26]:
#train_df = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/train.csv')
test_df = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/test.csv')
sample_submission = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/sample_submission.csv')

train_img_path = '/content/MyDrive/MyDrive/BoAZ/image/train/'
test_img_path = '/content/MyDrive/MyDrive/BoAZ/image/test/'

#train_df.head(),len(train_df)

In [32]:
train_df = train_df[train_df['image_id'].isin(train_images_indrive)]
train_df.head()

,ID,image_id,question,answer
0,TRAIN_000000,train_000000,Is this in the wild or zoo?,zoo
1,TRAIN_000003,train_000003,What are the walls made from?,drywall
2,TRAIN_000004,train_000004,How many players are sitting on the bench?,8
3,TRAIN_000006,train_000006,Where is the landing gear?,under plane
4,TRAIN_000007,train_000007,What colors are the sheets?,red


In [34]:
train_df = filtered_df.drop_duplicates(
    subset=["image_id", "question", "answer"]
).reset_index(drop=True)
train_df.shape

(111085, 4)

In [60]:
train_df = train_df[:100]

In [61]:
len(train_df)

100

## Dataset

In [7]:
class VQADataset(Dataset):
    def __init__(self, df, img_path, is_test=False):
        self.df = df
        self.processor = processor
        self.img_path = img_path
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        image = Image.open(self.img_path + row["image_id"] + ".jpg").convert('RGB')

        question = row['question']

        encoding = self.processor(
            image, question,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=text_len,
            return_tensors="pt"
        )

        if not self.is_test:

            answer = row['answer']

            targets = self.processor.tokenizer(
                answer,
                add_special_tokens=True,
                padding="max_length",
                truncation=True,
                max_length=text_len,
                return_tensors="pt"
            )

            encoding["labels"] = targets["input_ids"]

        for k,v in encoding.items():
            encoding[k] = v.squeeze()

        return encoding

In [3]:
if args.vision_language_pretrained_model == "Salesforce/blip-vqa-capfilt-large":
    vl_model_name = "Salesforce/blip-vqa-capfilt-large"
processor = AutoProcessor.from_pretrained(vl_model_name)
vl_model = BlipForQuestionAnswering.from_pretrained(vl_model_name)



In [4]:
class VQAModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.vl_model = vl_model

    def forward(self, inputs):
        outputs = self.vl_model(**inputs)
        return outputs

In [5]:
class VQAClassifier(pl.LightningModule):
    def __init__(self, backbone, args):
        super().__init__()
        self.backbone = backbone

    def forward(self, batch):
        predictions = self.backbone(batch)
        return predictions

    def step(self, batch):
        y = batch["labels"]
        outputs = self.backbone(batch)
        loss = outputs.loss
        y_hat = self.backbone.vl_model.generate(**batch)
        return loss, y, y_hat

    def compute_accuracy(self, y_hat, y):
        preds = []
        for i in range(len(y)):
            pred = processor.decode(y_hat[i], skip_special_tokens=True)
            true = processor.decode(y[i], skip_special_tokens=True)
            preds.append(pred==true)
        acc = np.sum(preds) / len(preds)
        return acc

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        acc = self.compute_accuracy(y_hat, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        acc = self.compute_accuracy(y_hat, y)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        acc = self.compute_accuracy(y_hat, y)
        self.log("test_acc", acc)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        y_hat = self.backbone.vl_model.generate(**batch)
        return y_hat

    def configure_optimizers(self):
        if args.optimizer == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=args.learning_rate, momentum=0.9)
        if args.optimizer == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=args.learning_rate)
        if args.optimizer == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=args.learning_rate)

        if args.scheduler == "none":
            return optimizer
        if args.scheduler == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer=optimizer,
                step_size=2,
                gamma=0.9,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "onecyclelr":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer=optimizer,
                max_lr=args.learning_rate,
                epochs=EPOCHS,
                steps_per_epoch=int(len(train_index) / BATCH_SIZE),
                pct_start=0.1,
            )
            return [optimizer], [scheduler]

In [35]:
train_df=train_df[:1100]

In [36]:
## preprocessing.py

temp_df, val_df = train_test_split(train_df, test_size=VALIDATION_SIZE, random_state=SEED)

temp_df = temp_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

## data_loaders.py

train_ds = VQADataset(temp_df, train_img_path, is_test=False)
val_ds = VQADataset(val_df, train_img_path, is_test=False)
#test_ds = VQADataset(test_df, test_img_path, is_test=True)

train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=args.num_workers)
val_dataloader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)
#test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)

## train.py

model = VQAClassifier(VQAModel(), args)

callbacks = [
    pl.callbacks.ModelCheckpoint(
        dirpath="./saved/", filename=f"{idx}",
        monitor="val_acc", mode="max",
    ),
]

trainer = pl.Trainer(
    max_epochs=EPOCHS, accelerator="auto", callbacks=callbacks,
    precision=args.mixed_precision, #logger=wandb_logger,
    devices=args.device
)

trainer.fit(model, train_dataloader, val_dataloader)

ckpt = torch.load(f"./saved/{idx}.ckpt", map_location=torch.device(device))
model.load_state_dict(ckpt['state_dict'])

## test.py

# eval_dict = trainer.validate(model, dataloaders=val_dataloader)[0]
# # wandb.log({'val_accuracy': eval_dict["val_acc"]})

# y_preds = trainer.predict(model, dataloaders=test_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type     | Params
--------------------------------------
0 | backbone | VQAModel | 384 M 
--------------------------------------
384 M     Trainable params
0         Non-trainable params
384 M     Total params
1,538.690 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=5` reached.


<All keys matched successfully>

In [10]:
just100_test_df=test_df[:100]
just100_sample_submission=sample_submission[:100]

In [37]:
# callbacks = [
#     pl.callbacks.ModelCheckpoint(
#         dirpath="./saved/", filename=f"{idx}",
#         monitor="val_acc", mode="max",
#     ),
# ]
# trainer = pl.Trainer(
#     max_epochs=EPOCHS, accelerator="auto", callbacks=callbacks,
#     precision=args.mixed_precision, #logger=wandb_logger,
#     devices=args.device
# )
test_ds = VQADataset(just100_test_df, test_img_path, is_test=True)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)
model = VQAClassifier(VQAModel(), args)

y_preds = trainer.predict(model, dataloaders=test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [38]:
outputs = []
for y_pred in y_preds:
    for output in y_pred:
        outputs.append(processor.decode(output, skip_special_tokens=True))

In [39]:
just100_sample_submission['answer'] = outputs
just100_sample_submission.to_csv('blip1_zeroshot_all.csv', index=False)

In [105]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!nvidia-smi

Mon Aug 21 08:40:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    34W /  70W |   4107MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [103]:
torch.cuda.empty_cache()

In [104]:
import gc
gc.collect()

0

In [99]:
train_df[:100]

,ID,image_id,question,answer
0,TRAIN_000000,train_000000,Is this in the wild or zoo?,zoo
1,TRAIN_000003,train_000003,What are the walls made from?,drywall
2,TRAIN_000004,train_000004,How many players are sitting on the bench?,8
3,TRAIN_000006,train_000006,Where is the landing gear?,under plane
4,TRAIN_000007,train_000007,What colors are the sheets?,red
...,...,...,...,...
95,TRAIN_000336,train_000335,What color is the dragon's tail?,green
96,TRAIN_000344,train_000343,What is the cat laying on?,laptop
97,TRAIN_000348,train_000347,How many shades of brown?,3
98,TRAIN_000354,train_000353,What color is dominant?,brown


In [116]:
test_df = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/test.csv')
test_df=test_df[:100]
test_ds = VQADataset(test_df, test_img_path, is_test=True)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)
eval_dict = trainer.validate(model, dataloaders=val_dataloader)[0]
# wandb.log({'val_accuracy': eval_dict["val_acc"]})

y_preds = trainer.predict(model, dataloaders=test_dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │            0.0            │
│         val_loss          │    1.7055528163909912     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [119]:
sample_submission=sample_submission[:100]

In [120]:
sample_submission['answer'] = outputs
sample_submission.to_csv(f'{idx}.csv', index=False)

In [ ]:
path = "/content/test_00001.jpg"
image = Image.open(path).convert('RGB')
# training
text = "what is she doing?"
label = "playing tennis"
inputs = processor(images=image, text=text, return_tensors="pt")
labels = processor(text=label, return_tensors="pt").input_ids

inputs["labels"] = labels
outputs = model(**inputs)
loss = outputs.loss
loss.backward()

# inference
text = "what is she doing?"
inputs = processor(images=image, text=text, return_tensors="pt")
outputs = model.generate(**inputs)
print(processor.decode(outputs[0], skip_special_tokens=True))

## Blip2

In [25]:
import pandas as pd
test_df = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/test.csv')
sample_submission = pd.read_csv('/content/MyDrive/MyDrive/BoAZ/sample_submission.csv')

train_img_path = '/content/MyDrive/MyDrive/BoAZ/image/train/'

test_img_path = '/content/MyDrive/MyDrive/BoAZ/image/test/'

test_df.head()

,ID,image_id,question
0,TEST_00000,test_00000,Is this a trade show?
1,TEST_00001,test_00001,Where are the people?
2,TEST_00002,test_00002,What color is the ski suit?
3,TEST_00003,test_00003,What color are the cabinets?
4,TEST_00004,test_00004,Is there a person standing in the background?


In [4]:
test_df['question'][0][:2]

'Is'

In [27]:
open_ended_df = pd.DataFrame()
open_ended_df

""


In [ ]:
for i in test_df['question']:
  if i[:2] != 'Is' or i[:2]!="Are":
    open_ended_df.append()

In [5]:
yesorno = ["Is", "Are"]
test_df = test_df[~test_df['question'].isin(yesorno)]
#filtered_df.reset_index(inplace=True,drop=True)
test_df.head()
len(test_df)
#filtered_df.to_csv(path_or_buf="without_closed_ended_question_index.csv")

40479

In [28]:
just100_test_df=test_df[:100]
just100_sample_submission=sample_submission[:100]

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")
model.to(device)

final_submission = just100_sample_submission.copy()

for idx, row in tqdm(just100_test_df.iterrows()):
    img_name = os.path.join(test_img_path, row['image_id'] + '.jpg')  # 이미지
    image = Image.open(img_name).convert('RGB')

    prompt =  "Question : {} answer:".format(row['question'])
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    generated_text = convert_word_to_number(generated_text)

    final_submission.loc[final_submission['ID'] == row['ID'], 'answer'] = generated_text

    if idx%100 == 0:
        final_submission.to_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/submission_zeroshot_blip2.csv', index=False)

final_submission.to_csv('/content/drive/MyDrive/Dacon_multimodal_for_zeroshot/submission_zeroshot_blip2_no_adjust.csv', index=False)